In [ ]:
# Sleep-EDF Dataset ile Uyku Evresi Sınıflandırması
# CNN-Transformer + GCN Mimarisi (EpiGraphNet)
print("Sleep Stage Classification with EpiGraphNet (Transformer Version)")
print("Dataset: Sleep-EDF (PhysioNet)")
print("Classes: W (Wake), N1, N2, N3 (SWS), REM")

/content


In [ ]:
# Gerekli kütüphaneleri yükle
!pip install -r requirements.txt -q

Cloning into 'epigraphnet'...
remote: Enumerating objects: 696, done.
remote: Counting objects: 100% (696/696), done.
remote: Compressing objects: 100% (639/639), done.
remote: Total 696 (delta 97), reused 652 (delta 53), pack-reused 0 (from 0)
Receiving objects: 100% (696/696), 3.92 MiB | 2.55 MiB/s, done.
Resolving deltas: 100% (97/97), done.


In [ ]:
# Sleep-EDF veri setini indir (otomatik)
from data.download import ensure_dataset
data_path = ensure_dataset(verbose=True)
print(f"Veri seti: {data_path}")

remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 7 (delta 3), reused 7 (delta 3), pack-reused 0 (from 0)
Unpacking objects: 100% (7/7), 8.51 KiB | 2.84 MiB/s, done.
From https://github.com/0nur0duncu/epigraphnet
   629f852..45903c4  main       -> origin/main
Updating 629f852..45903c4
Fast-forward
 main.ipynb               | 313 ++++++++++++++++++++++++++++++++++++++++----
 run_experiments_kfold.py | 334 +++++++++++++++++++++++++++++++++++++++++++++++
 2 files changed, 623 insertions(+), 24 deletions(-)
 create mode 100644 run_experiments_kfold.py


In [ ]:
# Veri setini yükle ve incele
from data import load_sleep_edf_dataset, get_sleep_stage_name
import numpy as np

signals, labels, _ = load_sleep_edf_dataset(
    max_subjects=5,  # Hızlı test için 5 özne
    verbose=True
)
print(f"\nSinyal şekli: {signals.shape}")
print(f"Her epoch: 30 saniye, 100 Hz = 3000 örnek")

/content/epigraphnet


In [ ]:
# Model mimarisini test et
import torch
from models import EpiGraphNet, BaselineCNNTransformer

# Test input (batch=2, channel=1, samples=3000 for 30s at 100Hz)
test_input = torch.randn(2, 1, 3000)

# EpiGraphNet test
model = EpiGraphNet(num_classes=5)
output = model(test_input)
print(f"EpiGraphNet output shape: {output.shape}")

# Baseline test
baseline = BaselineCNNTransformer(num_classes=5)
output = baseline(test_input)
print(f"BaselineCNNTransformer output shape: {output.shape}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.1 MB/s eta 0:00:00


In [ ]:
# Checkpoint dizinini temizle (isteğe bağlı)
import shutil
import os
if os.path.exists("checkpoints"):
    shutil.rmtree("checkpoints")
    print("Checkpoints temizlendi.")
os.makedirs("checkpoints", exist_ok=True)

In [ ]:
# Deneyleri çalıştır
# --max-subjects: Hızlı test için özne sayısını sınırla
# --num-runs: Her model için tekrar sayısı
# --channel: EEG kanalı (EEG Fpz-Cz veya EEG Pz-Oz)

!python run_all_experiments.py --num-runs 3 --max-subjects 10


######################################################################
# Sınıflandırma Türü: İkili (Binary)
######################################################################
Veri yüklendi: 500 örnek, 2 sınıf

Model: CNN-LSTM
    Early stopping at epoch 26 (best epoch: 11, best val_acc: 100.00%)                              
    Run 1 completed - Test Acc: 94.00%, F1: 90.96%                                                  
    Early stopping at epoch 28 (best epoch: 13, best val_acc: 98.00%)                               
    Run 2 completed - Test Acc: 92.00%, F1: 88.34%                                                  
    Early stopping at epoch 24 (best epoch: 9, best val_acc: 98.00%)                                
    Run 3 completed - Test Acc: 92.00%, F1: 85.12%                                                  
    Early stopping at epoch 40 (best epoch: 25, best val_acc: 100.00%)                              
    Run 4 completed - Test Acc: 98.00%, F1: 96.75%            

In [ ]:
# Sonuçları göster
import json

with open("results.json", "r") as f:
    results = json.load(f)

print("\n" + "="*60)
print("SONUÇLAR")
print("="*60)

for model_name, metrics in results["experiments"].items():
    print(f"\n{model_name}:")
    print(f"  Accuracy: {metrics['accuracy']:.2f}% ± {metrics['accuracy_std']:.2f}%")
    print(f"  F1 Score: {metrics['f1']:.2f}% ± {metrics['f1_std']:.2f}%")

Cihaz: cuda

######################################################################
# Sınıflandırma Türü: İkili (Binary)
# Yöntem: 10-Fold Stratified Cross Validation
######################################################################
Veri yüklendi: 500 örnek, 2 sınıf
Sınıf dağılımı: {np.int64(0): np.int64(400), np.int64(1): np.int64(100)}

Model: CNN-LSTM (10-Fold CV)
    Early stopping at epoch 31 (best: 11)                                                           
    Fold 1 - Test Acc: 100.00%, F1: 100.00%                                                         
    Early stopping at epoch 32 (best: 12)                                                           
    Fold 2 - Test Acc: 94.00%, F1: 91.53%                                                           
    Early stopping at epoch 32 (best: 12)                                                           
    Fold 3 - Test Acc: 96.00%, F1: 93.22%                                                           
    Early stopping 

In [ ]:
# Confusion matrix görselleştirme
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from data import load_sleep_edf_dataset, SleepEDFDataset
from models import EpiGraphNet
import torch
from torch.utils.data import DataLoader

# Örnek confusion matrix için
stage_names = ['W', 'N1', 'N2', 'N3', 'REM']
print("Uyku evreleri: ", stage_names)